In [91]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

import holidays

from sklearn.metrics import mean_squared_error


In [92]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")
external_data_cleaned = pd.read_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/grouped_data.csv")

In [93]:
df_train.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585


In [96]:
external_data_cleaned = external_data_cleaned[['date', 'T']]

In [97]:
external_data_cleaned.head(10)

,date,T
0,2020-09-01 00:00:00,12.933333
1,2020-09-01 01:00:00,12.683333
2,2020-09-01 02:00:00,12.250000
3,2020-09-01 03:00:00,12.133333
4,2020-09-01 04:00:00,11.616667
5,2020-09-01 05:00:00,11.400000
6,2020-09-01 06:00:00,11.533333
7,2020-09-01 07:00:00,13.283333
8,2020-09-01 08:00:00,15.016667
9,2020-09-01 09:00:00,17.400000


In [98]:
filtered_df_train = df_train[df_train['counter_name'] == "28 boulevard Diderot O-E"]

# Display the filtered DataFrame
filtered_df_train = filtered_df_train.sort_values(by='date', ascending=True)

filtered_df_train.head(10)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
57884,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2020-09-01 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
57887,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,2.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.098612
57890,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,7.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.079442
58933,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,10.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.397895
58936,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,48.0,2020-09-01 05:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,3.891820
58939,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,95.0,2020-09-01 06:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4.564348
57981,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,154.0,2020-09-01 07:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,5.043425
58942,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,58.0,2020-09-01 08:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4.077537
57893,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,45.0,2020-09-01 09:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,3.828641
57984,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,63.0,2020-09-01 10:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4.158883


In [99]:
# we convert 'date' column to datetime in all datasets
external_data_cleaned['date'] = pd.to_datetime(external_data_cleaned['date'])
df_train['date'] = pd.to_datetime(df_train['date']).astype('datetime64[ns]')
df_test['date'] = pd.to_datetime(df_test['date']).astype('datetime64[ns]')

# Sort datasets by date
external_data_cleaned.sort_values('date', inplace=True)
df_train.sort_values('date', inplace=True)
df_test.sort_values('date', inplace=True)

# we merge the data together  by performing the nearest match in terms of date :
training_set_merged = df_train.merge(external_data_cleaned, left_on="date", right_on="date", how='left')
testing_set_merged = df_test.merge(external_data_cleaned, left_on="date", right_on="date", how='left')

In [100]:
filtered_data = training_set_merged[training_set_merged['counter_name'] == "28 boulevard Diderot O-E"]
filtered_data = filtered_data.sort_values(by='date', ascending=True)

# Display the filtered DataFrame
filtered_data.head(10)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,T
0,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2020-09-01 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,12.683333
90,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,2.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.098612,12.250000
161,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,7.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.079442,12.133333
186,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,10.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.397895,11.616667
257,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,48.0,2020-09-01 05:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,3.891820,11.400000
297,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,95.0,2020-09-01 06:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4.564348,11.533333
368,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,154.0,2020-09-01 07:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,5.043425,13.283333
382,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,58.0,2020-09-01 08:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4.077537,15.016667
451,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,45.0,2020-09-01 09:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,3.828641,17.400000
520,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,63.0,2020-09-01 10:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,4.158883,18.683333


In [101]:
# Extract the date feature on different time scales :
fr_holidays = holidays.France()

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Add a feature to indicate if the day is a holiday in France
    X["is_holiday"] = X["date"].apply(lambda d: 1 if d in fr_holidays else 0)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

training_set_merged = _encode_dates(training_set_merged)
testing_set_merged = _encode_dates(testing_set_merged)


In [102]:
training_set_merged.head()

,counter_id,counter_name,site_id,site_name,bike_count,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,T,year,month,day,weekday,hour,is_weekend,is_holiday
0,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,12.683333,2020,9,1,1,1,0,0
1,100056226-103056226,Face au 8 avenue de la porte de Charenton SE-NO,100056226,Face au 8 avenue de la porte de Charenton,1.0,2019-11-01,"48.830331,2.400551",Y2H19070370,48.830331,2.400551,0.693147,12.683333,2020,9,1,1,1,0,0
2,100047545-104047545,Face 104 rue d'Aubervilliers S-N,100047545,Face 104 rue d'Aubervilliers,1.0,2018-11-29,"48.890457,2.368852",Y2H18086321,48.890457,2.368852,0.693147,12.683333,2020,9,1,1,1,0,0
3,100060178-102060178,90 Rue De Sèvres NE-SO,100060178,90 Rue De Sèvres,21.0,2020-07-22,"48.84638,2.31529",Y2H20052705,48.846380,2.315290,3.091042,12.683333,2020,9,1,1,1,0,0
4,100056327-103056327,Face au 4 avenue de la porte de Bagnolet E-O,100056327,Face au 4 avenue de la porte de Bagnolet,2.0,2019-11-06,"48.86461,2.40969",Y2H19070372,48.864610,2.409690,1.098612,12.683333,2020,9,1,1,1,0,0


In [103]:
training_set_merged.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'counter_installation_date', 'coordinates', 'counter_technical_id',
       'latitude', 'longitude', 'log_bike_count', 'T', 'year', 'month', 'day',
       'weekday', 'hour', 'is_weekend', 'is_holiday'],
      dtype='object')

In [104]:
training_set_merged.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'counter_installation_date', 'coordinates', 'counter_technical_id',
       'latitude', 'longitude', 'log_bike_count', 'T', 'year', 'month', 'day',
       'weekday', 'hour', 'is_weekend', 'is_holiday'],
      dtype='object')

In [105]:
'''
# Define bins and labels for temperature categories in Kelvin
bins = [-float('inf'), 278.15, 283, 298, 308.15, float('inf')]  # Updated Kelvin thresholds
labels = ['very_cold', 'cold', 'moderate', 'warm', 'very_hot']

# Create a new categorical feature for temperature
training_set_merged['temp_category'] = pd.cut(training_set_merged['temperature'], bins=bins, labels=labels)
testing_set_merged['temp_category'] = pd.cut(testing_set_merged['temperature'], bins=bins, labels=labels)

# One-hot encode the categories for the model
training_set_merged = pd.get_dummies(training_set_merged, columns=['temp_category'], drop_first=True)
testing_set_merged = pd.get_dummies(testing_set_merged, columns=['temp_category'], drop_first=True)

# remove temperature column :
training_set_merged = training_set_merged.drop(columns=['temperature'])
testing_set_merged = testing_set_merged.drop(columns=['temperature'])
'''

"\n# Define bins and labels for temperature categories in Kelvin\nbins = [-float('inf'), 278.15, 283, 298, 308.15, float('inf')]  # Updated Kelvin thresholds\nlabels = ['very_cold', 'cold', 'moderate', 'warm', 'very_hot']\n\n# Create a new categorical feature for temperature\ntraining_set_merged['temp_category'] = pd.cut(training_set_merged['temperature'], bins=bins, labels=labels)\ntesting_set_merged['temp_category'] = pd.cut(testing_set_merged['temperature'], bins=bins, labels=labels)\n\n# One-hot encode the categories for the model\ntraining_set_merged = pd.get_dummies(training_set_merged, columns=['temp_category'], drop_first=True)\ntesting_set_merged = pd.get_dummies(testing_set_merged, columns=['temp_category'], drop_first=True)\n\n# remove temperature column :\ntraining_set_merged = training_set_merged.drop(columns=['temperature'])\ntesting_set_merged = testing_set_merged.drop(columns=['temperature'])\n"

In [106]:
# training_set_merged.nunique()

In [107]:
X_train = training_set_merged.drop(columns=["bike_count", "log_bike_count"])
y_train = training_set_merged["log_bike_count"]

X_test = testing_set_merged.copy()

In [108]:
# Preprocessing pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', TableVectorizer()),
        ('model', xgb.XGBRegressor()),
    ]
)

In [109]:
# Fit Pipeline to Training Data
pipeline.fit(X_train, y_train)

# Make Predictions on Test Data
y_predictions = pipeline.predict(X_test)


In [110]:
print(y_predictions)

[0.19365884 1.4670876  1.6551778  ... 2.2597272  2.7220156  2.292259  ]


In [111]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_XGboost_vsimple_weather_newdata.csv", index=False)

In [112]:
# We can print the RMSE on the training data :
y_train_predictions = pipeline.predict(X_train)
rmse_train = mean_squared_error(y_train, y_train_predictions, squared=False)
print(f"Training RMSE: {rmse_train}")

Training RMSE: 0.4033123335672844


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [113]:
# code to get feature importance :


# Step 1: Extract the preprocessor and feature names
# Retrieve the preprocessor from the pipeline
preprocessor = pipeline.named_steps['preprocessor']

# Get the feature names after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Step 2: Extract the trained XGBoost model and feature importance
xgb_model = pipeline.named_steps['model']

# Get feature importances from the trained XGBoost model
feature_importance = xgb_model.feature_importances_

# Step 3: Combine feature names and importance scores into a DataFrame
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Sort features by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display top features
print("Top Features by Importance:")
importance_df.head(40)

Top Features by Importance:


,Feature,Importance
40,"counter_name: cours, reine, totem",0.144331
93,counter_installation_date_day,0.101828
52,"counter_name: austerlitz, 85, no",0.100874
10,"counter_id: 102007049, 101007049, 100007049",0.084270
13,"counter_id: 100056331, 104056331, 103056331",0.062229
162,hour,0.051008
15,"counter_id: 106056336, 100056336, sc",0.039310
39,"counter_name: sébastopol, 73, de",0.030723
12,"counter_id: 103057445, 100057445, 104057445",0.017963
33,"counter_name: orsay, quai, 67",0.015663
